<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       BreuschGodfrey Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>BreuschGodfrey</b></p>
<p style = 'font-size:16px;font-family:Arial'>The BreuschGodfrey() function checks for the presence of serial correlation among the residual and error terms after running a regression associated with a fitted model. With respect to regression models, it is expected that there is no serial correlation among the error terms.</p>

<p style = 'font-size:16px;font-family:Arial'>The Breusch-Godfrey test determines if there is evidence of autocorrelation in the residuals of a model. Autocorrelation is correlation between the errors or residuals of a regression model meaning errors are not independent. The value of an error at a given point in time is related to the earlier error values.</p>

<p style = 'font-size:16px;font-family:Arial'>Breusch-Godfrey tests the hypothesis that there is or is not autocorrelation in the residuals. The test regresses the residuals on their lagged values for a number of lags, and then performing a chi-squared test on the residuals of the regression.</p>

<p style = 'font-size:16px;font-family:Arial'>If autocorrelation is present in the residuals of a regression model, the standard errors of the estimates of the regression coefficients may be biased, and lead to incorrect inference and predictions. Using Breusch-Godfrey in time-series analysis and econometrics, you can identify potential problems with you regression model and correct them.</p>

<p style = 'font-size:16px;font-family:Arial'> The following procedure is an example of how to use BreuschGodfrey:</p>
<li style = 'font-size:16px;font-family:Arial'>Use LinearRegr() for regression testing.</li>
<li style = 'font-size:16px;font-family:Arial'>Use BreuschGodfrey() on the result from LinearRegr() to compute the test statistics and determine if there is serial correlation.</li>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame, 
    in_schema,
    TDSeries,
    LinearRegr,
    BreuschGodfrey,
    db_drop_table,
    db_drop_view,
    remove_context
    )

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_BreuschGodfrey.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["house_values"])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data = DataFrame.from_table("house_values")
data

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. BreuschGodfrey</b>
<p style = 'font-size:16px;font-family:Arial'>The BreuschGodfrey() function checks for the presence of serial correlation among the residual and error terms after running a regression associated with a fitted model. With respect to regression models, it is expected that there is no serial correlation among the error terms.</p>

<p></p>
<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(BreuschGodfrey)

<p style = 'font-size:16px;font-family:Arial'>We need to first convert the data from dataframe into a TDSeries which will be passed to the DickeyFuller function as input.</p>

In [ ]:
data_series_df = TDSeries(data=data,
                              id="cityid",
                              row_index="TD_TIMECODE",
                              row_index_style="TIMECODE",
                              payload_field=["salary","house_val"],
                              payload_content="MULTIVAR_REAL")

In [ ]:
filter_expr = data_series_df.cityid==33
linear_regr_result = LinearRegr(data=data_series_df,
                                data_filter_expr=filter_expr,
                                formula = "Y = B0 + B1*X1",
                                weights=False,
                                algorithm='QR',
                                coeff_stats=True,
                                variables_count=2,
                                model_stats=True,
                                residuals=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will compute the serial correlation among the residual and error terms. Input is teradataml TDSeries object over the "fitresiduals" dataframe generated by running a linear regression associated with a fitted model.</p>

In [ ]:
data_series_bg = TDSeries(data=linear_regr_result.fitresiduals,
                              id="cityid",
                              row_index="ROW_I",
                              row_index_style= "SEQUENCE",
                              payload_field=["RESIDUAL","ACTUAL_VALUE","CALC_VALUE"],
                              payload_content="MULTIVAR_REAL")

In [ ]:
uaf_out = BreuschGodfrey(data=data_series_bg,
                             explanatory_count=2,
                             residual_max_lags=1,
                             significance_level=.01)
 
# Print the result DataFrame.
uaf_out.result

<p></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A returned NULL_HYPOTH value of ACCEPT means no serial correlation. A returned NULL_HYPOTH value of REJECT means there is evidence of serial correlation.
</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_table("house_values")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>BreuschGodfrey function reference: <a href = 'https://docs.teradata.com/search/all?query=BreuschGodfrey&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>